In [ ]:
# Server Side
import io
import os
import tqdm
import glob
import traceback
import psycopg2
import pymongo
import gridfs
import bson
from bson.objectid import ObjectId
from flask import Flask
from flask import request
from zipfile import ZipFile
from multiprocessing.pool import ThreadPool

from embeddings import get_embeddings


class MongoDB:
    def __init__(self, cred):
        self.mongo_cred = cred
        self.mongo_client = pymongo.MongoClient(self.mongo_cred)
        self.mongo_db = self.mongo_client.dummy
        self.mongo_col = self.mongo_db["data"]
        self.mongo_fs = gridfs.GridFS(self.mongo_db,collection='full_images')
    
    def put_to_fs(self, im_bytes, meta=False):
        if meta:
            return self.mongo_fs.put(im_bytes, filename=meta["image_name"], meta=meta, encoding=('utf-8')).__str__()
        return self.mongo_fs.put(im_bytes, encoding=('utf-8')).__str__()


class PostGresDB:
    def __init__(self, database, user, password, host, port):
        self.conn = psycopg2.connect(
           database=database, user=user, password=password, host=host, port=port
        )
        self.conn.autocommit = True
        self.cur = self.conn.cursor()
        
    def insert_record(self, name, embedding, path):
        self.cur = self.conn.cursor()
        sql_query = '''insert into image_embeddings(name, embeddings, path)
        values(%s, %s, %s) RETURNING id; '''
        self.cur.execute(sql_query,(name, embedding, path))
        return self.cur.fetchone()[0]


mongo_db_obj = MongoDB('127.0.0.1:27017')
postgres_db_obj = PostGresDB(database='nouman', user='nouman', password='nomi351492', host='localhost', port='5432')

def handle_image(im_path):
    global mongo_db_obj
    global postgres_db_obj
    try:
        name = os.path.basename(im_path).split(".")[0]
        im_vector = get_embeddings(im_path)
        data = pickle.dumps(im_vector)
        key_value = postgres_db_obj.insert_record(name, data, im_path)
        data_to_store = {"id":key_value, "image_name":os.path.basename(im_path)}
        with open(im_path, "rb") as f:
            id_ = mongo_db_obj.put_to_fs(f.read(), meta=data_to_store)
            return id_
    except:
        print(str(traceback.format_exc()))
        pass
    return False
    


zips_dest_dirs = "extracted_zips"
if not os.path.exists(zips_dest_dirs):
    os.mkdir(zips_dest_dirs)


app = Flask(__name__)
@app.route('/enroll', methods=['GET', 'POST'])
def enroll():
    try:
        f = request.files["archive"]
        zip_file_name = f.filename.split(".")[0]
        myzip = ZipFile(io.BytesIO(f.read()))
        myzip.extractall(zips_dest_dirs)
        dir_name = False
        dir_names = [z for z in myzip.infolist() if z.is_dir()]
        if dir_names:
            dir_name = dir_names[0].filename
            paths_list = glob.glob(os.path.join(zips_dest_dirs, dir_name,"**","*.*"), recursive=True)
            lt = []
            with ThreadPool(50) as p:
                lt.append(list(tqdm.tqdm(p.imap(handle_image, paths_list), total=len(paths_list))))
            return {"ststus": str(len([s for s in lt[0] if s]))+" images done"}
    except Exception as e:
        print(str(traceback.format_exc()))
        return {"status":str(traceback.format_exc())}


app.run(host='0.0.0.0', port=5000)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.16.10.146:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Mar/2023 11:59:58] "POST / HTTP/1.1" 404 -


In [2]:
# Server Side
import io
import os
import tqdm
import glob
import traceback
import psycopg2
import pymongo
import gridfs
import bson
from bson.objectid import ObjectId
from flask import Flask
from flask import request
from zipfile import ZipFile
from multiprocessing.pool import ThreadPool
#from embeddings import get_embeddings


class MongoDB:

    def init(self, cred):
        self.mongo_cred = cred

        self.mongo_client = pymongo.MongoClient(self.mongo_cred)

        self.mongo_db = self.mongo_client.dummy

        self.mongo_col = self.mongo_db["data"]

        self.mongo_fs = gridfs.GridFS(self.mongo_db,collection='full_images')


def put_to_fs(im_bytes, meta=False):
    pass# your code here

class PostGresDB:

      def init(self, database, user, password, host, port):

            self.conn = psycopg2.connect(

            database=database, user=user, password=password, host=host, port=port)

            self.conn.autocommit = True

            self.cur = self.conn.cursor()


def insert_record(self, name, embedding, path):
    sql_query = '''insert into image_embeddings(name, embedding, path)
    values(%s, %s, %s) RETURNING id; '''
    #CREATE TABLE Image_Embeddings;
    # your code here
def get_embeddings(im_name):
    pass
def handle_image(im_path, mongo_db_obj, postgres_db_obj):
    try:
        im_name = os.path.basename(im_path).split(".")[0]

# your code here, get vector using get_embeddings(im_name) and store in data variable

        key_value = postgres_db_obj.insert_record(name, data, im_path)

        data_to_store = {"id":key_value, "image_name":im_name}

        id_ = mongo_db_obj.put_to_fs(im_bytes, meta=data_to_store)
        return id_
    except:
        pass

    return False


#mongo_db_obj = MongoDB('your credentials')

postgres_db_obj = PostGresDB(database='nouman', user='nouman', password='nomi351492', host='localhost', port= "5432")


zips_dest_dirs = "extracted_zips"

if not os.path.exists(zips_dest_dirs):
    os.mkdir(zips_dest_dirs)


    app = Flask(name)

@app.route('/enroll', methods=['GET', 'POST'])

def enroll():
    try:
        return {"status": str(no_of_inserted_images)+" images done"}

    except Exception as e:
        return {"status":str(traceback.format_exc())}


app.run(host='0.0.0.0', port=8000)

TypeError: PostGresDB() takes no arguments

In [2]:
#pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.1/492.1 kB 3.5 MB/s eta 0:00:002.3 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 16.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
